---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [21]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [22]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [23]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    data = pd.read_csv('university_towns.txt', header=None, sep='\t')

    mask_State = data[0].str.contains(r"\[edit\]")
    states = data[mask_State]
    states = states[0].str.replace(r"\[edit\]", "").str.strip()
    data["State"] = states

    mask_RegionName = ~mask_State
    RegionName = data[mask_RegionName]
    RegionName = RegionName[0].str.replace(r"\s\(.*", "")
    data["RegionName"] = RegionName
    
    # Drop original column
    data = data.drop(0, axis=1)
    
    states = data[["State"]].fillna(method='ffill')
    data["State"] = states

    # Delete row where RegionName is NaN
    data.dropna(axis=0, how='any', inplace=True)
    
    data = data.reset_index(drop=True)

    return data


get_list_of_university_towns()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [24]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    quarters = df['Unnamed: 4']
    gdp = df['Unnamed: 6']
    quarters = list(quarters[218:]) #including 1999q4 to calculate growth for 2000q1
    gdp = list(gdp[218:])  #including 1999q4 to calculate growth for 2000q1
    growth = []
    for i in range(1,len(gdp)):
        growth.append(gdp[i]-gdp[i-1])
    quarters = quarters[1:]  #removing 1999q4 data 
    gdp = gdp[1:] #removing 1999q4 data 
    ret = ""
    for i in range(1,len(growth)-1):
        if growth[i]<0 and growth[i+1]<0:
            ret = quarters[i]
            break
    return ret
get_recession_start()

'2008q3'

In [25]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    quarters = df['Unnamed: 4']
    gdp = df['Unnamed: 6']
    quarters = list(quarters[218:]) #including 1999q4 to calculate growth for 2000q1
    gdp = list(gdp[218:])  #including 1999q4 to calculate growth for 2000q1
    growth = []
    for i in range(1,len(gdp)):
        growth.append(gdp[i]-gdp[i-1])
    quarters = quarters[1:]  #removing 1999q4 data 
    gdp = gdp[1:] #removing 1999q4 data 
    found_index = -1
    ret = ""
    for i in range(1,len(growth)-1):
        if growth[i]<0 and growth[i+1]<0:
            found_index = i
            break
    for i in range(found_index+2,len(growth)-1):
        if growth[i]>0 and growth[i+1]>0:
            ret = quarters[i+1]
            break
    return ret
get_recession_end()

'2009q4'

In [26]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    quarters = df['Unnamed: 4']
    gdp = df['Unnamed: 6']
    quarters = list(quarters[219:])
    gdp = list(gdp[219:])
    start = quarters.index(get_recession_start())
    end = quarters.index(get_recession_end())
    ret = start
    for i in range(start,end+1):
        if int(gdp[i])<int(gdp[ret]):
            ret = i
    return quarters[ret]
get_recession_bottom()


'2009q2'

In [27]:
def helper(element):
    if element<10:
        return "0"+str(element)
    else:
        return str(element)
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    cleaning_list = ['RegionID','Metro','CountyName','SizeRank']
    year = 1996
    month = 4
    while year<2000:
        for i in range(month,13):
            if i<=9:
                cleaning_list.append(str(year)+"-0"+str(i))
            else:
                cleaning_list.append(str(year)+"-"+str(i))
        month = 1
        year = year + 1
    df = df.drop(cleaning_list,axis = 1)
    for qt in range(1,5):
        for yr in range(2000,2017):
            if qt==1:
                try:
                    df[str(yr)+"q"+str(qt)] = df[[str(yr)+"-"+helper(1),str(yr)+"-"+helper(2),str(yr)+"-"+helper(3)]].mean(axis=1)
                    df = df.drop([str(yr)+"-"+helper(1),str(yr)+"-"+helper(2),str(yr)+"-"+helper(3)],axis=1)
                except:
                    continue
            elif qt==2:
                try:
                    df[str(yr)+"q"+str(qt)] = df[[str(yr)+"-"+helper(4),str(yr)+"-"+helper(5),str(yr)+"-"+helper(6)]].mean(axis=1)
                    df = df.drop([str(yr)+"-"+helper(4),str(yr)+"-"+helper(5),str(yr)+"-"+helper(6)],axis=1)
                except:
                    continue
            elif qt==3:
                try:
                    df[str(yr)+"q"+str(qt)] = df[[str(yr)+"-"+helper(7),str(yr)+"-"+helper(8),str(yr)+"-"+helper(9)]].mean(axis=1)
                    df = df.drop([str(yr)+"-"+helper(7),str(yr)+"-"+helper(8),str(yr)+"-"+helper(9)],axis=1)
                except:
                    continue
            else:
                try:
                    df[str(yr)+"q"+str(qt)] = df[[str(yr)+"-"+helper(10),str(yr)+"-"+helper(11),str(yr)+"-"+helper(12)]].mean(axis=1)
                    df = df.drop([str(yr)+"-"+helper(10),str(yr)+"-"+helper(11),str(yr)+"-"+helper(12)],axis=1)
                except:
                    continue
    df['2016q3'] = df[['2016-07','2016-08']].mean(axis=1)
    df = df.drop(['2016-07','2016-08'],axis=1)
    for i in range(0,len(df)):
        df.ix[i,'State'] = str(states[df.iloc[i]['State']])
    df = df.set_index(['State','RegionName'])
    return df
convert_housing_data_to_quarters()

C:\Users\user10\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,,2000q1,2001q1,2002q1,2003q1,2004q1,2005q1,2006q1,2007q1,2008q1,2009q1,...,2007q4,2008q4,2009q4,2010q4,2011q4,2012q4,2013q4,2014q4,2015q4,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,4.226667e+05,4.881333e+05,5.172333e+05,5.085000e+05,4.777333e+05,...,5.133667e+05,4.879333e+05,4.583667e+05,4.772667e+05,4.637000e+05,4.691667e+05,4.924333e+05,5.280667e+05,5.728333e+05,587200.0
California,Los Angeles,2.070667e+05,2.330000e+05,2.619667e+05,3.189000e+05,3.952333e+05,4.950667e+05,5.817667e+05,5.917667e+05,5.353000e+05,4.262667e+05,...,5.613667e+05,4.439667e+05,4.043333e+05,4.020667e+05,3.755667e+05,3.965667e+05,4.719333e+05,5.188667e+05,5.577333e+05,584050.0
Illinois,Chicago,1.384000e+05,1.569333e+05,1.755000e+05,1.871000e+05,2.062000e+05,2.239000e+05,2.424000e+05,2.472333e+05,2.437333e+05,2.237667e+05,...,2.468333e+05,2.270333e+05,2.116667e+05,1.913667e+05,1.702000e+05,1.672333e+05,1.858667e+05,2.012667e+05,2.079000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.533333e+04,5.913333e+04,6.473333e+04,7.373333e+04,8.970000e+04,1.083333e+05,1.173333e+05,1.195667e+05,1.162000e+05,...,1.208000e+05,1.158667e+05,1.185667e+05,1.177667e+05,1.147000e+05,1.108333e+05,1.124667e+05,1.156667e+05,1.222000e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.196000e+05,1.265333e+05,1.357000e+05,1.437000e+05,1.742333e+05,2.428000e+05,2.430000e+05,2.186333e+05,1.775667e+05,...,2.289333e+05,1.833333e+05,1.434667e+05,1.158667e+05,1.031667e+05,1.304333e+05,1.599000e+05,1.685000e+05,1.838333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.395333e+05,1.493333e+05,1.594667e+05,1.900667e+05,2.690000e+05,2.954333e+05,2.899667e+05,2.509333e+05,1.812000e+05,...,2.649333e+05,1.949333e+05,1.361333e+05,1.238000e+05,1.104000e+05,1.247667e+05,1.625667e+05,1.754667e+05,1.906333e+05,199950.0
California,San Diego,2.229000e+05,2.672000e+05,3.012333e+05,3.664333e+05,4.400667e+05,5.312333e+05,5.415000e+05,5.128333e+05,4.619667e+05,3.957000e+05,...,4.811667e+05,4.076333e+05,3.985667e+05,3.896667e+05,3.665667e+05,4.019667e+05,4.664667e+05,4.964333e+05,5.254667e+05,539750.0
Texas,Dallas,8.446667e+04,8.973333e+04,9.256667e+04,9.820000e+04,1.027000e+05,1.140333e+05,1.178667e+05,1.213333e+05,1.199000e+05,1.076667e+05,...,1.231333e+05,1.099000e+05,1.034333e+05,9.833333e+04,9.383333e+04,9.770000e+04,1.045333e+05,1.115333e+05,1.346000e+05,149300.0
California,San Jose,3.742667e+05,4.706667e+05,4.414333e+05,4.782667e+05,5.049667e+05,6.052667e+05,6.851333e+05,6.962667e+05,6.430667e+05,5.481667e+05,...,6.714000e+05,5.601000e+05,5.065333e+05,5.142333e+05,4.882000e+05,5.511000e+05,6.456333e+05,7.149333e+05,7.891333e+05,822200.0


In [28]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    data = convert_housing_data_to_quarters().copy()
    unv_towns = get_list_of_university_towns()
    start = get_recession_start()
    end = get_recession_end()
    bottom = get_recession_bottom()
    before_recession_column = ""
    if int(start[-1])==1:
        before_recession_column = str(int(start[:4])-1)+"q4"
    else:
        before_recession_column = start[:5] + str(int(start[-1])-1)
    data['price_ratio'] = data[before_recession_column] / data[bottom]
    data = data.reset_index()
    data = data[['State','RegionName','price_ratio']]
    unv_towns_data = pd.merge(data,unv_towns,how='inner',on=['State','RegionName'])
    nunv_towns_data = pd.merge(data,unv_towns,how='outer',on=['State','RegionName'])
    unv_towns_data = unv_towns_data.dropna()
    nunv_towns_data = nunv_towns_data.dropna()
    ret_pvalue = (ttest_ind(nunv_towns_data['price_ratio'],unv_towns_data['price_ratio']).pvalue)
    ret_better = ""
    if unv_towns_data['price_ratio'].mean()<nunv_towns_data['price_ratio'].mean():
        ret_better = "university town"
    else:
        ret_better = "non-university town"
    return (ret_pvalue<0.01,ret_pvalue,ret_better)
run_ttest()

C:\Users\user10\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


(True, 0.003386197848363605, 'university town')